In [1]:
import requests
import pandas as pd
import mysql.connector
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options as ChromeOptions

In [ ]:
lista_productos = []  # Lista para almacenar los productos más baratos de todos los supermercados

url = 'https://www.cotodigital3.com.ar/sitios/cdigi/browse?Dy=1&Nf=product.startDate%7CLTEQ+1.6901568E12%7C%7Cproduct.endDate%7CGTEQ+1.6901568E12&Nr=AND%28product.sDisp_200%3A1004%2Cproduct.language%3Aespañol%2Cproduct.siteId%3ACotoDigital%29&Nty=1&_D%3AidSucursal=+&_D%3AsiteScope=+&atg_store_searchInput=s&idSucursal=200&siteScope=ok'

# Realizar la solicitud HTTP a la página
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Obtener el contenido HTML de la página
    html_content = response.text

    # Crear un objeto BeautifulSoup para analizar el contenido HTML
    soup = BeautifulSoup(html_content, "html.parser")

    # Encontrar el elemento deseado por su id
    elemento = soup.find("span", {"id": "resultsCount"})

    # Verificar si se encontró el elemento
    if elemento:
        # Extraer el valor del elemento
        valor_elemento = elemento.text.strip()

    cant_elementos = float(valor_elemento)

while cant_elementos != 0:
    # Restantes es desde donde tenemos que mostrar
    restantes = 0
    
    # Cantidad de productos que podemos mostrar por iteracion
    limite_mostrar = 1000

    lista_productos_categoria = []  # Lista para almacenar los productos más baratos del supermercado actual

    # Configurar el driver de Selenium y cargar la página web
    options = ChromeOptions()
    options.add_argument("--start-minimized")
    driver = webdriver.Chrome(options=options)
    
    supermercado = 'https://www.cotodigital3.com.ar/'
        
    driver.get(f'{supermercado}sitios/cdigi/browse?Dy=1&Nf=product.startDate%7CLTEQ+1.6901568E12%7C%7Cproduct.endDate%7CGTEQ+1.6901568E12&No={restantes}&Nr=AND%28product.sDisp_200%3A1004%2Cproduct.language%3Aespañol%2Cproduct.siteId%3ACotoDigital%2COR%28product.siteId%3ACotoDigital%29%29&Nrpp={limite_mostrar}&Nty=1&_D%3AidSucursal=+&_D%3AsiteScope=+&atg_store_searchInput=s&idSucursal=200&siteScope=ok')

    super = supermercado.split('.')[1].upper()

    # Esperar hasta que el elemento sea visible
    wait = WebDriverWait(driver, 7)
    
    try:
        elemento = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span.atg_store_newPrice')))
    except:
        continue

    # Obtener el contenido de la página web con BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Obtenemos todos los elementos contenedores de productos
    productos = soup.find_all('div', 
                                {'class': 'valtech-carrefourar-product-summary-status-0-x-container valtech-carrefourar-product-summary-status-0-x-productNotAdded flex flex-column h-100'})

    for producto in productos:
        try:
            # Precio con Tarjeta Carrefour
            precio = producto.find('span', 
                                {'class': 'valtech-carrefourar-product-price-0-x-currencyContainer'}).text

            # Precio sin Tarjeta Carrefour
            #precio = producto.find('span', 
                                #{'class': 'valtech-carrefourar-product-price-0-x-listPriceValue strike'}).text
        except AttributeError:
            continue
        
        # Guardamos el nombre de los productos
        nombre = producto.find('h3', 
                                {'class': 'vtex-product-summary-2-x-productNameContainer mv0 vtex-product-summary-2-x-nameWrapper overflow-hidden c-on-base f5'}).span.text
        
        try:
            # Promo Carrefour %OFF
            promo = producto.find('span', 
                                    {'class': 'valtech-carrefourar-product-price-0-x-discountPercentage f6 lh-copy'}).text.strip()
        except AttributeError:
            try:
                # Promo Carrefour con tarjeta Carrefour
                promo = producto.find('span', 
                                        {'class': 'valtech-carrefourar-product-highlights-0-x-productHighlightText'}).text
            
                # A revisar, agregar una nueva columna de precio sin promocion, en caso de que se necesite usar tarjeta para la misma.
            
            except AttributeError:
                try:
                    # Promo Carrefour con tarjeta Carrefour
                    promo = producto.find('span', 
                                            {'class': 'valtech-carrefourar-product-highlights-0-x-productHighlightText'}).text
                    
                    # Promo Carrefour Adicional
                    promo2 = producto.find('span', 
                                            {'class': 'valtech-carrefourar-product-highlights-0-x-productHighlightText'}).span.text

                    promo += promo2
                    
                except AttributeError:
                    try:
                        # Promo Carrefour Adicional
                        promo = producto.find('span', 
                                                {'class': 'valtech-carrefourar-product-highlights-0-x-productHighlightText'}).span.text
                    except AttributeError:
                            promo = 'No tiene promo'
                            
        if 'MI CRF' in promo:
            precio_sin_promo = producto.find('span', 
                                        {'class': 'valtech-carrefourar-product-price-0-x-listPriceValue strike'}).text
        else:
            precio_sin_promo = precio
        
        url = producto.find('a', 
                            {'class': 'vtex-product-summary-2-x-clearLink'})['href']

        url.split('/')[1]

        precios = [precio, precio_sin_promo]

        for precios_int in precios:
            precios_int = precios_int.replace('.', '')
            precios_int = precios_int.replace(',', '.')
            precios_int = precios_int.replace('$', '')
            precios_int = float(precios_int)

        producto_dict = {
            'supermercado': super,
            'nombre': nombre,
            'precio': precio,
            'promo': promo,
            'url': supermercado + url,
            'precio sin promo': precio_sin_promo
        }

        lista_productos_categoria.append(producto_dict)

    lista_productos.extend(lista_productos_categoria)
    
    if cant_elementos >= 1000:
        restantes += 1000
        cant_elementos -= 1000
    else:
        cant_elementos = 0
